In [1]:
from sub_func import *
from pf_selection import *
from pipeline_utils import *

from t1 import *
from t2_t4 import *
from t5 import *

/opt/anaconda3/envs/finTF/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


예시로, 2023년 1분기 분석을 위해 2022년 4분기 정보를 바탕으로 2023년 1분기 초기 포트폴리오 생성

In [2]:
# 조회할 정보의 기간
base_year = '2023'
base_quarter = 'Q3'
start_date = '20230701'
end_date = '20231031'

# 포트폴리오를 활용하는 실제 기간 (ex. 2022 Q4를 조회하여 2023 Q1을 위한 포트폴리오 생성)
target_year = '2023'
target_quarter = 'Q4'

# 오늘 날짜 (코스피 장이 열린 날만 한정, 예시)
kospi_open_days = get_kospi_open_days("20231001", "20231231")

# 'pf_selection_agent'에 초기 포트폴리오가 있는지 확인
db_info = get_all_page_ids_from_database('pf_selection_agent')
init_pf_page_title = f"{target_year}_{target_quarter}_init_pf"

if init_pf_page_title not in list(db_info.keys()):
    print('초기 포트폴리오를 생성합니다...')
    pf_selection_info = pf_selection_main(base_year, base_quarter, start_date, end_date)
    print("노션에 output을 모두 저장합니다...")
    to_DB('pf_selection_agent', init_pf_page_title, f"{start_date}_{end_date}", str(pf_selection_info))

# 'pf_selection_agent'에서 종목 코드 가져오기
temp_tickers = get_tickers_from_json_v2('pf_selection_agent', f'{target_year}_{target_quarter}_init_pf')
tickers = []

for ticker in temp_tickers:
    if not is_delisted(ticker, today):
        tickers.append(ticker)
        
weight_dict = {ticker: 1 / len(tickers) for ticker in tickers}
 
# JSON 파일 저장할 디렉토리 경로
log_dir = f"./pf_logs/{target_year}_{target_quarter}"
os.makedirs(log_dir, exist_ok=True)  # 디렉토리 없으면 생성

# 파일 경로
file_path = os.path.join(log_dir, f"{target_year}_{target_quarter}_init_pf.json")

with open(f'{file_path}', "w", encoding="utf-8") as f:
    json.dump(weight_dict, f, indent=2, ensure_ascii=False)

In [3]:
tickers = temp_tickers

tickers

['042700',
 '195870',
 '004710',
 '007810',
 '308170',
 '000040',
 '000270',
 '001450']

In [4]:
def finTF(today, tickers, base_year, base_quarter, target_year, target_quarter, start_date):

    print('***** t1_analyst_main 실행 *****')
    t1_analyst_main(today, tickers, base_year, base_quarter)

    print('***** t1_pf_manager_main 실행 *****')
    t1_pf_manager_main(today, tickers, base_year, base_quarter, target_year, target_quarter)

    print('***** t1_trader_main 실행 *****')
    t1_trader_main(today, tickers, base_year, base_quarter)

    print('***** t2_t4_main 실행 *****')
    t_2_t_4_main(tickers, start_date, today, target_year, target_quarter, base_year, base_quarter)

    print('***** t5_main 실행 *****')
    t_5_main(today, base_year, base_quarter)

In [14]:
kospi_open_days[49]

'20231213'

In [15]:
test_span = kospi_open_days[49:]
# 1026부터 실행

for today in test_span:
    finTF(today, tickers, base_year, base_quarter, target_year, target_quarter, start_date)
    print(f'{today}일자 실행 완료')

***** t1_analyst_main 실행 *****
[t1_analyst] === 종목별 분석 시작 ===
UK에 대해 Inflation Rate 정보를 찾을 수 없습니다. | [Errno 2] No such file or directory: '/Users/yeonsuk/investment/finTF/pipeline/sub_func/get_info/../../../store_data/raw/FRED/UK/Inflation Rate/2023/2023_Inflation Rate.csv'
JP에 대해 Inflation Rate 정보를 찾을 수 없습니다. | [Errno 2] No such file or directory: '/Users/yeonsuk/investment/finTF/pipeline/sub_func/get_info/../../../store_data/raw/FRED/JP/Inflation Rate/2023/2023_Inflation Rate.csv'

=== 042700 분석 중... ===
analyze_financial_data의 GPT 요청 결과: {'id': 'chatcmpl-Ay6fwXkr0lN2OYJp8Rwgh8m2LIlzP', 'choices': [{'finish_reason': 'stop', 'index': 0, 'logprobs': None, 'message': {'content': '# 재무 분석 보고서\n\n## 1. 재무제표 요약\n\n### 1.1. 재무상태표 (제 44 기 3분기 말)\n- **자산총계**: 636,436억원 (전년 동기 대비 39.7% 증가)\n- **유동자산**: 242,269억원 (전년 동기 대비 9.0% 감소)\n- **비유동자산**: 394,167억원 (전년 동기 대비 112.3% 증가)\n- **부채총계**: 79,146억원 (전년 동기 대비 20.9% 증가)\n- **자본총계**: 557,290억원 (전년 동기 대비 42.8% 증가)\n\n### 1.2. 손익계산서 (제 44 기 3분기)\n- *

/Users/yeonsuk/investment/finTF/pipeline/t1.py:470: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  corp_news_df['SA_result'] = corp_news_df['news_title'].apply(lambda x:



=== 195870 분석 중... ===
analyze_financial_data의 GPT 요청 결과: {'id': 'chatcmpl-Ay6qjxSsBnTyrLaZp5V8HWMzKT7Rs', 'choices': [{'finish_reason': 'stop', 'index': 0, 'logprobs': None, 'message': {'content': '# 재무제표 분석 보고서\n\n## 1. 재무제표 요약\n\n### 1.1 자산\n- 자산총계: 681,773,364,000 KRW (2023년 3분기)\n- 유동자산: 360,721,686,000 KRW, 감소 (2022년 12월: 371,321,313,000 KRW)\n- 비유동자산: 321,051,678,000 KRW, 증가 (2022년 12월: 288,292,509,000 KRW)\n\n### 1.2 부채\n- 유동부채: 152,404,579,000 KRW, 감소 (2022년 12월: 196,470,095,000 KRW)\n- 비유동부채: 22,904,311,000 KRW, 증가 (2022년 12월: 15,848,433,000 KRW)\n- 부채총계: 175,308,890,000 KRW, 감소 (2022년 12월: 212,318,528,000 KRW)\n\n### 1.3 자본\n- 자본총계: 506,464,474,000 KRW, 증가 (2022년 12월: 447,295,294,000 KRW)\n- 이익잉여금: 399,082,232,000 KRW, 증가 (2022년 12월: 340,510,401,000 KRW)\n\n## 2. 주요 재무 비율 분석\n- **주가(PER)**: 56,200\n- **PER**: 0.0000557 (매우 낮음)\n- **PBR**: 0.0000019 (매우 낮음)\n- **ROE**: 3.39% (상대적으로 낮은 수익성)\n- **Profit Growth Rate**: -66.64% (상당한 감소)\n- **CAGR**: -0.67% (부정적 성장)\n\n## 3. 재무적 

/Users/yeonsuk/investment/finTF/pipeline/t1.py:470: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  corp_news_df['SA_result'] = corp_news_df['news_title'].apply(lambda x:



=== 004710 분석 중... ===
analyze_financial_data의 GPT 요청 결과: {'id': 'chatcmpl-Ay6rRWu0K49q1exoCMCO365LplwSO', 'choices': [{'finish_reason': 'stop', 'index': 0, 'logprobs': None, 'message': {'content': '# 재무제표 분석 보고서\n\n## 1. 요약\n\n2023년 3분기 연결 재무상태표를 기준으로 주요 재무 지표를 분석했습니다. 연결 자산 총계는 1,060,284백만원으로 전년 동기 대비 감소했으며, 이는 유동자산의 감소와 비유동자산의 소폭 증가에 기인합니다. 특히, 유동자산의 주요 구성 요소인 현금 및 현금성 자산이 크게 감소했습니다.\n\n## 2. 주요 재무정보\n\n### 2.1 자산\n\n- **자산총계**: 1,060,284백만원 (전기: 1,071,922백만원)\n- **유동자산**: 491,608백만원 (전기: 509,416백만원)\n- **비유동자산**: 568,676백만원 (전기: 562,506백만원)\n\n**분석**: \n유동자산이 감소한 이유는 현금 및 현금성 자산이 67,406백만원으로 전기 92,139백만원에서 감소했기 때문입니다. 이는 단기 자산 유동성에 부정적인 영향을 미칠 수 있습니다.\n\n### 2.2 부채\n\n- **부채총계**: 569,790백만원 (전기: 631,896백만원)\n- **유동부채**: 375,647백만원 (전기: 424,709백만원)\n- **비유동부채**: 194,143백만원 (전기: 207,187백만원)\n\n**분석**: \n부채 총계의 감소는 유동부채의 감소에 기인합니다. 이는 단기 채무 상환능력이 개선되었다고 볼 수 있습니다.\n\n### 2.3 자본\n\n- **자본총계**: 490,494백만원 (전기: 440,026백만원)\n- **자본금**: 160,549백만원 (변동 없음)\n- **이익잉여금**: 124,127백만원 (전기: 78,4

...

'Timestamp' | Error occurred at 000040
=== 7/8 ===


python(88041) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(88042) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(88043) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(88044) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(88045) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(88046) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(88048) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(88049) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(88050) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(88051) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(88052) Malloc

=== 8/8 ===


python(88191) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(88192) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(88193) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(88194) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(88195) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(88196) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(88197) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(88198) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(88199) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(88200) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(88201) Malloc

=== [9] 1/8 (042700) ===
=== [9] 2/8 (195870) ===
=== [9] 3/8 (004710) ===
=== [9] 4/8 (007810) ===
=== [9] 5/8 (308170) ===
=== [9] 6/8 (000040) ===
=== [9] 7/8 (000270) ===
=== [9] 8/8 (001450) ===
=== [10] 1/8 (042700) ===
=== [10] 2/8 (195870) ===
=== [10] 3/8 (004710) ===
=== [10] 4/8 (007810) ===
=== [10] 5/8 (308170) ===
=== [10] 6/8 (000040) ===
=== [10] 7/8 (000270) ===
=== [10] 8/8 (001450) ===
=== [11] 1/8 (042700) ===
=== [11] 2/8 (195870) ===
=== [11] 3/8 (004710) ===
=== [11] 4/8 (007810) ===
=== [11] 5/8 (308170) ===
=== [11] 6/8 (000040) ===
=== [11] 7/8 (000270) ===
=== [11] 8/8 (001450) ===
=== [12] 1/8 (042700) ===
=== [12] 2/8 (195870) ===
=== [12] 3/8 (004710) ===
=== [12] 4/8 (007810) ===
=== [12] 5/8 (308170) ===
=== [12] 6/8 (000040) ===
=== [12] 7/8 (000270) ===
=== [12] 8/8 (001450) ===
=== [13] 1/8 (042700) ===
=== [13] 2/8 (195870) ===
=== [13] 3/8 (004710) ===
=== [13] 4/8 (007810) ===
=== [13] 5/8 (308170) ===
=== [13] 6/8 (000040) ===
=== [13] 7/8 (000270

'20230116'